In [1]:
####Load the packages

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import string
import nltk
import it_core_news_sm
import re
from itertools import combinations
import itertools
import os
import networkx as nx

from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
#from nltk.tokenize import word_tokenize
from nltk.tokenize import WhitespaceTokenizer
word_tokenize = WhitespaceTokenizer()
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from wordcloud import WordCloud

#### Load the lemmas file

In [2]:
database_words = pd.read_csv("id_lemmas_posts.csv",index_col = 0 , sep=',', engine='python')
database_words['text_nlp'] = database_words.apply(lambda row: word_tokenize.tokenize(row['text_nlp']), axis = 1)
database_words

,text_nlp
0,"[musulmano, pensare, portare, guerra, santo, c..."
1,"[particolare, ringraziamento, magistrato, pale..."
2,"[minaccia, insulto, scontro, piazzare, bloccar..."
3,"[pensiero, radical, chic, sinistro, ribrezzo, ..."
4,"[verme, protagonista, ultimo, violenza, donna,..."
...,...
10098,"[viaggiare, strasburgo, agenda, plenario, xadi..."
10099,"[conferenza, programmatico, federazione, provi..."
10100,"[eccellenza, italiano, famiglia, ravizza, pavi..."
10101,"[italiano, favorevole, castrazione, chimico, p..."


#### Posts database

In [3]:
database_posts = pd.read_csv('database/postDB.csv', engine='python')
database_posts.head(2)

,Unnamed: 0,Origin_file_order,Site,p_id,dateCreated,p_politician,p_gender,p_GRUPPO_PE,p_LISTA,p_PARTITO,...,p_SemiC,p_Qmark,p_Exclam,p_Dash,p_Quote,p_Apostro,p_Parenth,p_OtherP,Count,target1_2
0,0,30126,FB,96844400700_10157493758850701,2019-04-23T10:33:37Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,NaN
1,15,53320,FB,911961728894076_2169530686470501,2019-04-17T12:50:26Z,IGOR GELARDA,M,EFDD,LEGA,Lega,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,NaN


## Subset
**select the subset - p_rating, p_camapagna2**

**p_rating - This is the level of hate of the post (positive/ negative/ problematic /ambiguo)**

**p_camapagna2 - Type of campaign Neg-comp, Positiva***


In [4]:
subset_posts = database_posts.loc[:,['p_rating', 'p_camapagna2']]
p_camapagna2 = subset_posts.p_camapagna2.unique().tolist()

In [5]:
subset_posts

,p_rating,p_camapagna2
0,problematico,Neg-comp
1,positivo,Neg-comp
2,positivo,Neg-comp
3,positivo,Neg-comp
4,negativo,Neg-comp
...,...,...
10098,positivo,Positiva
10099,positivo,Positiva
10100,positivo,Positiva
10101,positivo,Positiva


In [6]:
p_camapagna2

['Neg-comp', 'Positiva']

## obtain the file with nodes+attributes and edges

In [7]:
folder_path = 'posts_subsets/'
os.makedirs(folder_path,exist_ok=True)

In [8]:
def get_id_from_node(word, ids):
    return ids.index[ids.label == word][0]

def get_subset_p_camapagna2(subset_posts_input, database_words_input):
    
    for p_camapagna2_single_value in p_camapagna2:
        
        print("Now p_camapagna2 ---->  ", p_camapagna2_single_value)
        #skip the nan values
        if p_camapagna2_single_value == 'nan': continue
        
        subset_posts = subset_posts_input[subset_posts_input.loc[:,'p_camapagna2'] == str(p_camapagna2_single_value)]
        if len(subset_posts) == 0: continue
        
        database_words = database_words_input.copy()
        
        matching_indexes = database_words.index.intersection(subset_posts.index)
        subset_words = database_words.loc[matching_indexes,:]
        df_words = subset_words.copy()
        df_words['id']      = df_words.index
        df_words['p_rating'] = subset_posts.p_rating
        
        unique_words = {}
        words_per_comment = {}
        
        words_problematico = {}
        words_positivo = {}
        words_negativo = {}
        words_ambiguo = {}
###
        for index, row in df_words.iterrows():
            if(len(row['text_nlp'])>0):
                for single_word in row['text_nlp']:
                    unique_words.setdefault(single_word, 0) ## setdefault() method returns the value of a key (if the key is in dictionary)
                    unique_words[single_word] += 1

                    if row['p_rating'] == 'problematico':
                        words_problematico.setdefault(single_word, 0) ## setdefault() method returns the value of a key (if the key is in dictionary)
                        words_problematico[single_word] += 1

                    elif row['p_rating'] == 'positivo':
                        words_positivo.setdefault(single_word, 0) ## setdefault() method returns the value of a key (if the key is in dictionary)
                        words_positivo[single_word] += 1

                    elif row['p_rating'] == 'negativo':
                        words_negativo.setdefault(single_word, 0) ## setdefault() method returns the value of a key (if the key is in dictionary)
                        words_negativo[single_word] += 1
                        
                    elif row['p_rating'] == 'ambiguo':
                        words_ambiguo.setdefault(single_word, 0) ## setdefault() method returns the value of a key (if the key is in dictionary)
                        words_ambiguo[single_word] += 1

        unique_words.pop('#', None)
        
        lemmas_df         = pd.DataFrame.from_dict(unique_words, orient='index').dropna().reset_index()
        lemmas_df         = lemmas_df.reset_index()
        lemmas_df.columns = ['id', 'label', 'count']
        lemmas_df         = lemmas_df.drop('count', axis = 1)
        
        lemmas_df['counts']      = lemmas_df.label.apply(lambda x: unique_words.get(x))
        lemmas_df['counts_problematico'] = lemmas_df.label.apply(lambda x: words_problematico.get(x) if x in words_problematico.keys() else 0)
        lemmas_df['counts_positivo'] = lemmas_df.label.apply(lambda x: words_positivo.get(x) if x in words_positivo.keys() else 0)
        lemmas_df['counts_negativo']  = lemmas_df.label.apply(lambda x: words_negativo.get(x) if x in words_negativo.keys() else 0)
        lemmas_df['counts_ambiguo']  = lemmas_df.label.apply(lambda x: words_ambiguo.get(x) if x in words_ambiguo.keys() else 0)
        
        ##save the nodes attributes
        lemmas_df.to_csv('posts_subsets/NODES_'+str(p_camapagna2_single_value) + '_prating.csv', index = False)

        words_per_comment = {}

        for index, row in df_words.iterrows():
            if(len(row['text_nlp'])>0):
                words_per_comment[row["id"]]= [get_id_from_node(clean_txt, lemmas_df) for clean_txt in row['text_nlp']
                                               if clean_txt!='#'] ###iterating the column text_clean. There are 78174 rows
              
        words_projection = {} ##create dictionary 
        ##itertools.product() which computes the cartesian product of input iterables.

        for key in words_per_comment:
            for pair in itertools.product(words_per_comment[key],words_per_comment[key]):
                if pair[0]!=pair[1] and not (pair[::-1] in words_projection):
                    words_projection.setdefault(pair,0)
                    words_projection[pair] += 1
                    
        words_weighted = []
        for edge in words_projection:
            words_weighted.append((edge[0],edge[1],words_projection[edge]))

        G = nx.Graph()
        G.add_weighted_edges_from(words_weighted)
        nx.write_weighted_edgelist(G, "posts_subsets/EDGES_"+str(p_camapagna2_single_value)+'_prating.csv',delimiter=",") ##save the edges list as csv
        ## save the edges list
        df_edges = pd.read_csv('posts_subsets/EDGES_'+str(p_camapagna2_single_value)+'_prating.csv')
        df_edges.columns = ['source', 'target', 'weight']
        df_edges.to_csv('posts_subsets/EDGES_'+str(p_camapagna2_single_value)+'_prating.csv', index = False)

        
get_subset_p_camapagna2(subset_posts, database_words)

Now p_camapagna2 ---->   Neg-comp
Now p_camapagna2 ---->   Positiva


## FOR ADJACENT WORDS DATASET obtain the file with nodes+attributes and edge

In [9]:
def get_id_from_node(word, ids):
    return ids.index[ids.label == word][0]

def get_subset_p_camapagna2_adjacent(subset_posts_input, database_words_input):
    
    for p_camapagna2_single_value in p_camapagna2:
        
        print("Now p_camapagna2 ---->  ", p_camapagna2_single_value)
        #skip the nan values
        if p_camapagna2_single_value == 'nan': continue
        
        subset_posts = subset_posts_input[subset_posts_input.loc[:,'p_camapagna2'] == str(p_camapagna2_single_value)]
        if len(subset_posts) == 0: continue
        
        database_words = database_words_input.copy()
        
        matching_indexes = database_words.index.intersection(subset_posts.index)
        subset_words = database_words.loc[matching_indexes,:]
        df_words = subset_words.copy()
        df_words['id']      = df_words.index
        df_words['p_rating'] = subset_posts.p_rating
        
        unique_words = {}

        for index, row in df_words.iterrows():
            if(len(row['text_nlp'])>0):
                for single_word in row['text_nlp']:
                    unique_words.setdefault(single_word, 0) ## setdefault() method returns the value of a key (if the key is in dictionary)
                    unique_words[single_word] += 1

        unique_words.pop('#', None)
        
        lemmas_df         = pd.DataFrame.from_dict(unique_words, orient='index').dropna().reset_index()
        lemmas_df         = lemmas_df.reset_index()
        lemmas_df.columns = ['id', 'label', 'count']
        lemmas_df         = lemmas_df.drop('count', axis = 1)
        

        words_per_comment = {}

        for index, row in df_words.iterrows():
            if(len(row['text_nlp'])>0):
                words_per_comment[row["id"]]= [get_id_from_node(clean_txt, lemmas_df) for clean_txt in row['text_nlp']
                                               if clean_txt!='#'] ###iterating the column text_clean. There are 78174 rows

                    
        words_projection = {} ##create dictionary 
        ##itertools.product() which computes the cartesian product of input iterables.
        
        for row in words_per_comment:
            for index, word in enumerate(words_per_comment.get(row)):
                #row is finished, therefore continue to next row, skip the last element that has index len(x)-1
                if index == (len(words_per_comment.get(row)) - 1):
                    continue
                row_words = words_per_comment.get(row).copy()
                #row is not finished, then see whether two consecutive words are different
                if word!=row_words[index+1] and not ((word,row_words[index+1])[::-1] in words_projection):
                    words_projection.setdefault((word,row_words[index+1]),0)
                    words_projection[(word,row_words[index+1])] += 1
                       
        words_weighted = []
        for edge in words_projection:
            words_weighted.append((edge[0],edge[1],words_projection[edge]))

        G = nx.Graph()
        G.add_weighted_edges_from(words_weighted)
        nx.write_weighted_edgelist(G, "posts_subsets/EDGES_"+str(p_camapagna2_single_value)+"_adjacent.csv",delimiter=",") ##save the edges list as csv
        df_edges = pd.read_csv('posts_subsets/EDGES_'+str(p_camapagna2_single_value)+'_adjacent.csv')
        df_edges.columns = ['source', 'target', 'weight']
        df_edges.to_csv('posts_subsets/EDGES_'+str(p_camapagna2_single_value)+'_adjacent.csv', index = False)

        
get_subset_p_camapagna2_adjacent(subset_posts, database_words)

Now p_camapagna2 ---->   Neg-comp
Now p_camapagna2 ---->   Positiva


In [10]:
filename_edges = ['posts_subsets/EDGES_'+str(p_camapagna2_edge)+'_prating.csv' for p_camapagna2_edge in p_camapagna2]
filename_edges

['posts_subsets/EDGES_Neg-comp_prating.csv',
 'posts_subsets/EDGES_Positiva_prating.csv']

## ADD SENTIMENT
##### p_rating attributes 

In [12]:
filename_nodes = ['posts_subsets/NODES_'+str(p_camapagna2_sing)+'_prating.csv' for p_camapagna2_sing in p_camapagna2]
filename_nodes


['posts_subsets/NODES_Neg-comp_prating.csv',
 'posts_subsets/NODES_Positiva_prating.csv']

In [13]:
subset_pos = pd.read_csv(filename_nodes[1])
subset_pos = subset_pos.dropna().reset_index(drop = True)
subset_pos

,id,label,counts,counts_problematico,counts_positivo,counts_negativo,counts_ambiguo
0,0,collegamento,52,0,51,1,0
1,1,continuo,66,0,61,5,0
2,2,tempo,167,1,153,13,0
3,3,pastorello,1,0,1,0,0
4,4,recuperare,12,0,12,0,0
...,...,...,...,...,...,...,...
12942,12942,#ricorso,1,0,1,0,0
12943,12943,xadiscussione,1,0,1,0,0
12944,12944,xadibattito,2,0,2,0,0
12945,12945,xaregole,1,0,1,0,0


In [14]:
subset_neg = pd.read_csv(filename_nodes[0])
subset_neg = subset_neg.dropna().reset_index(drop = True)
subset_neg

,id,label,counts,counts_problematico,counts_positivo,counts_negativo,counts_ambiguo
0,0,musulmano,15,5,3,6,1
1,1,pensare,113,7,54,52,0
2,2,portare,148,17,81,50,0
3,3,guerra,76,7,32,33,4
4,4,santo,18,2,6,10,0
...,...,...,...,...,...,...,...
10980,10981,quindicenne,1,1,0,0,0
10981,10982,#vigliacco,1,0,0,1,0
10982,10983,indugiare,1,0,1,0,0
10983,10984,brambilla,1,0,1,0,0


## Upload Sentiment dictionary
**See here for better comprehension of the sentiment dictionary**

In [15]:
sentix = pd.read_csv('sentix.file', sep = "\t", header = None, dtype = {'lemmas': object})
sentix.columns = ['lemma', 'POS', 'Wordnet synset ID' ,'positive score', 'negative score', 'polarity','intensity']
sentix = sentix.dropna()
sentix

,lemma,POS,Wordnet synset ID,positive score,negative score,polarity,intensity
0,abile,a,1740,0.125,0.00,1.0,0.125
1,intelligente,a,1740,0.125,0.00,1.0,0.125
2,valente,a,1740,0.125,0.00,1.0,0.125
3,capace,a,1740,0.125,0.00,1.0,0.125
4,incapace,a,2098,0.000,0.75,-1.0,0.750
...,...,...,...,...,...,...,...
74604,imbronciarsi,v,2771020,0.000,0.25,-1.0,0.250
74605,rannuvolarsi,v,2771020,0.000,0.25,-1.0,0.250
74606,rasserenarsi,v,2771169,0.125,0.00,1.0,0.125
74607,rischiararsi,v,2771169,0.125,0.00,1.0,0.125


### The next call to the function get_rows_data(word, sentix, use_similarity = False) above does the following:

- Output: Returns a row of the kind (lemma, positive score, negative score, polarity, intensity)

- What it does: it looks for a matching word in the sentix dataframe.
- If there is one exactly matching, returns that row if there are many, then compute the mean between the two positive and negative scores among the different rows, polarity is the sign of the difference (positive_score - negative_score) and intensity is the  𝐿2 L2 norm (i.e. sqrt(pos^2 + neg^2))

- If there is NONE exactly matching, then look for any string that matches a word in the sentix dictionary. Same reasoning as before. If there is only one matching then returns that row, otherwise do some algebra.

- If still there are none matching, then look for any string that contains (indeed to contain is a less strict relation that matching that in turn is a less strict relation than exactly match)

- If still there are none contained, if a flag is set to true, then take the most similar words in the lemmas list and use them to create sentiment.
- If even this way there are none, or the previous flag was set to false, return a row of the kind (word, 0. , 0. , 0.)

In [16]:
def get_polarity(pos, neg):
    
    if (pos == 0.)and(neg == 0.):
        return 0.
    
    elif pos != 0.:
        theta = np.arctan(neg/pos)
    
    elif pos == 0: 
        theta = np.arctan(np.Inf)
    
    return 1.-4.*theta/np.pi

def get_exact_match(word, sentix):
    subset = sentix[sentix.loc[:,'lemma'].str.fullmatch(word) == True]
    
    if (len(subset) == 0):
        return subset.loc[:,['lemma','positive score','negative score','polarity','intensity']]
    
    if (len(subset) == 1):
        return subset.loc[:,['lemma','positive score','negative score','polarity','intensity']]
        
    elif (len(subset) > 1):
        pos_mean  = subset.loc[:,'positive score'].mean()
        neg_mean  = subset.loc[:,'negative score'].mean()
        polarity  = get_polarity(pos_mean,neg_mean)
        intensity = np.sqrt(pos_mean**2 + neg_mean**2)
        
        dictionary = {'lemma': [word], 'positive score' : [pos_mean],'negative score' : [neg_mean],
                             'polarity' : [polarity],'intensity' : [intensity]}
        
        return pd.DataFrame.from_dict(dictionary)
    
def get_match(word, sentix):
    subset = sentix[sentix.loc[:,'lemma'].str.match(word) == True]
    
    if (len(subset) == 0):
        return subset.loc[:,['lemma','positive score','negative score','polarity','intensity']]
    
    if (len(subset) == 1):
        return subset.loc[:,['lemma','positive score','negative score','polarity','intensity']]
        
    elif (len(subset) > 1):
        pos_mean  = subset.loc[:,'positive score'].mean()
        neg_mean  = subset.loc[:,'negative score'].mean()
        polarity  = get_polarity(pos_mean,neg_mean)
        intensity = np.sqrt(pos_mean**2 + neg_mean**2)
        
        dictionary = {'lemma': [word], 'positive score' : [pos_mean],'negative score' : [neg_mean],
                             'polarity' : [polarity],'intensity' : [intensity]}
        
        return pd.DataFrame.from_dict(dictionary)
    
def get_contains(word, sentix):
    subset = sentix[sentix.loc[:,'lemma'].str.contains(word) == True]
    
    if (len(subset) == 0):
        return subset.loc[:,['lemma','positive score','negative score','polarity','intensity']]
    
    if (len(subset) == 1):
        return subset.loc[:,['lemma','positive score','negative score','polarity','intensity']]
        
    elif (len(subset) > 1):
        pos_mean  = subset.loc[:,'positive score'].mean()
        neg_mean  = subset.loc[:,'negative score'].mean()
        polarity  = get_polarity(pos_mean,neg_mean)
        intensity = np.sqrt(pos_mean**2 + neg_mean**2)
        
        dictionary = {'lemma': [word], 'positive score' : [pos_mean],'negative score' : [neg_mean],
                             'polarity' : [polarity],'intensity' : [intensity]}
        
        return pd.DataFrame.from_dict(dictionary)
    
def get_similar(word, sentix):
    similar_words = difflib.get_close_matches(word, sentix.lemma, n = 8, cutoff = 0.8)
    subset = sentix[sentix.loc[:,'lemma'].isin(similar_words) == True]
    
    if (len(subset) == 0):
        return subset.loc[:,['lemma','positive score','negative score','polarity','intensity']]
    
    if (len(subset) == 1):
        return subset.loc[:,['lemma','positive score','negative score','polarity','intensity']]
        
    elif (len(subset) > 1):
        pos_mean  = subset.loc[:,'positive score'].mean()
        neg_mean  = subset.loc[:,'negative score'].mean()
        polarity  = get_polarity(pos_mean ,neg_mean)
        intensity = np.sqrt(pos_mean**2 + neg_mean**2)
        
        dictionary = {'lemma': [word], 'positive score' : [pos_mean],'negative score' : [neg_mean],
                             'polarity' : [polarity],'intensity' : [intensity]}
        
        return pd.DataFrame.from_dict(dictionary)

#instead of returning a row of a dictionary, return a list of its values
def denest_row(row):
    
    return [row.loc[:,'lemma'].values[0], 
            row.loc[:,'positive score'].values[0], 
            row.loc[:,'negative score'].values[0], 
            row.loc[:,'polarity'].values[0],
            row.loc[:,'intensity'].values[0]]

def get_rows_data(word, sentix, use_similarity = False):
    
    row = get_exact_match(word, sentix)
    
    #if there are no exact matches, try others
    if (len(row) == 0):
        print('no exact match')
        row = get_match(word, sentix)
        
        #if there are no matches, try the others
        if (len(row) == 0):
            print('no match')
            row = get_contains(word, sentix)
            
            #if word is not even contained and we want similarities then try it
            if ((len(row) == 0) and (use_similarity)):
                print('no contains')
                row = get_similar(word, sentix)
            
                #if still no matches, give up and return an empty dataframe
                if (len(row) == 0):
                    print('no similar')
                    dictionary = {'lemma': [word], 'positive score' : [0.],'negative score' : [0.],
                                  'polarity' : [0.],'intensity' : [0.]}
                    row =  pd.DataFrame.from_dict(dictionary)
            
            #if we do not want similarities then return the empty dataframe        
            elif (len(row) == 0):
                print('no contains')
                dictionary = {'lemma': [word], 'positive score' : [0.],'negative score' : [0.],
                                  'polarity' : [0.],'intensity' : [0.]}
                row = pd.DataFrame.from_dict(dictionary)
    
    row = row.reset_index(drop = True)
    #get back the original word    
    row.lemma = word
    
    #compute the right polarity
    if (row.loc[0,'positive score'] != 0.):
        row.polarity = get_polarity(row.loc[0,'positive score'] , row.loc[0,'negative score'] )
        


    #denest it
    return denest_row(row)

In [17]:
labels = ['lemma', 'positive_score', 'negative_score', 'polarity', 'intensity']

In [18]:
###Load the posts lemmas
words = pd.read_csv('Posts_lemmas_unique.csv')
words = words.dropna().reset_index(drop = True)
words = words.dropna(how = 'all')
words.columns = ['id','lemma']
words

,id,lemma
0,0,musulmano
1,1,pensare
2,2,portare
3,3,guerra
4,4,santo
...,...,...
17576,17577,#ricorso
17577,17578,xadiscussione
17578,17579,xadibattito
17579,17580,xaregole


In [13]:
sentiment_list = [] 
# for word in words.iloc[0:100,0]:
for i, word in enumerate (words.iloc[:,0]):
    print(type(i))
    if ( i % 50 == 0): print("word #", i+1, "/", len(words))
    try:
      sentiment_list.append(get_rows_data(word, sentix, True)) 
    except KeyError:
      print("key error", word, i)

<class 'int'>
word # 1 / 17581
no exact match
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
no exact match
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
<class 'int'>
<class 'int'>
<class 'int'>
no exact match
no match
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
no exact match
<class 'int'>
<class 'int'>
no exact match
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
<class 'int'>
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
<class 'int'>
no exact match
no match
no contains
<class 'int'>
<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
no similar
<cl

<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
<class 'int'>
no exact match
no match
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
<class 'int'>
<class 'int'>
<class 'int'>
no exact match
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
<class 'int'>
<class 'int'>
<class 'int'>
word # 301 / 17581
<class 'int'>
<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
<class 'int'>
<class 'int'>
no exact match
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact m

no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
<class 'int'>
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
<class 'int'>
no exact match
no match
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
word # 551 / 17581
no exact match
no match
no contains
no similar
<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
<class 'int'>
no exact match
no match
no contains
<class 'int'>
<class 'int'>
no exact match
no match
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
no exact match
no match
<class 'int'>
<class 'int'>
<class 'int'>
no exact match
no matc

no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
<class 'int'>
no exact match
<class 'int'>
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
<class 'int'>
no exact match
no match
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
<class 'int'>
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
<class 'int'>
no exact match
no match
<class 'int'>
<class 'int'>
<class 'int'>
no exact match
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
word # 801 / 17581
<class 'int'>
no exact match
<

<class 'int'>
<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
<class 'int'>
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match

...

no similar
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
<class 'int'>
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
n

<class 'int'>
no exact match
<class 'int'>
no exact match
no match
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no sim

<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
<class 'int'>
word # 17151 / 17581
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no

no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
<class 'int'>
no exact match
no match
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
<class 'int'>
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no c

no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no match
no contains
<class 'int'>
no exact match
no match
<class 'int'>
no exact match
no match
no contains
no similar
<class 'int'>
no exact match
no matc

**Add Sentiment to pcampagna**

In [14]:
sentiment_dataframe = pd.DataFrame.from_records(sentiment_list, columns = labels)
sentiment_dataframe

,lemma,positive_score,negative_score,polarity,intensity
0,musulmano,0.2500,0.000000,1.000000,0.250000
1,pensare,0.1875,0.083333,0.467500,0.205185
2,portare,0.1250,0.062500,0.409666,0.139754
3,guerra,0.0625,0.125000,-0.409666,0.139754
4,santo,0.7500,0.000000,1.000000,0.750000
...,...,...,...,...,...
17576,#ricorso,0.1375,0.300000,-0.452810,0.330009
17577,xadiscussione,0.1250,0.083333,0.251332,0.150231
17578,xadibattito,0.1250,0.000000,1.000000,0.125000
17579,xaregole,0.0000,0.000000,0.000000,0.000000


In [15]:
sentiment_dataframe.to_csv('Posts_lemmas_sentiment.csv', index = False)

##### Load the posts sentiment file

In [19]:
####load the posts with the sentiment value

posts = pd.read_csv('Posts_lemmas_sentiment.csv')
posts = posts.dropna().reset_index(drop = True)
posts = posts.dropna(how = 'all')
# posts.columns = ['id','lemma']
posts

,lemma,positive_score,negative_score,polarity,intensity
0,musulmano,0.2500,0.000000,1.000000,0.250000
1,pensare,0.1875,0.083333,0.467500,0.205185
2,portare,0.1250,0.062500,0.409666,0.139754
3,guerra,0.0625,0.125000,-0.409666,0.139754
4,santo,0.7500,0.000000,1.000000,0.750000
...,...,...,...,...,...
17576,#ricorso,0.1375,0.300000,-0.452810,0.330009
17577,xadiscussione,0.1250,0.083333,0.251332,0.150231
17578,xadibattito,0.1250,0.000000,1.000000,0.125000
17579,xaregole,0.0000,0.000000,0.000000,0.000000


In [20]:
##polarity for pos_pcampagna
###12947
polarity_list = []

for i in range(len(subset_pos)):
    if (i%2000 == 0): print(i, "/", len(subset_pos))
    word = subset_pos.label[i]    
    polarity_list.append(posts[posts.loc[:,'lemma'] == word].polarity.tolist()[0])
polarity_list

0 / 12947
2000 / 12947
4000 / 12947
6000 / 12947
8000 / 12947
10000 / 12947
12000 / 12947


[0.2513318327560048,
 -0.8730979302777859,
 1.0,
 0.0,
 -1.0,
 0.0,
 -0.5431989951203675,
 0.0,
 -0.8274407551461038,
 1.0,
 -0.4959169809045496,
 0.0,
 -1.0,
 -0.22239955114314086,
 0.4096655293982669,
 1.0,
 1.0,
 0.0,
 -0.5431989951203675,
 0.7215820509077442,
 0.4903977680535604,
 0.01632268965010697,
 1.0,
 0.0,
 0.5155242336366264,
 1.0,
 -1.0,
 -0.688083478490523,
 -0.4096655293982672,
 0.0,
 1.0,
 0.563248354265482,
 1.0,
 0.4096655293982669,
 0.0,
 1.0,
 0.8730979302777857,
 -0.4616056596010187,
 1.0,
 0.5155242336366264,
 0.0,
 0.2513318327560048,
 1.0,
 0.5903344706017328,
 0.0,
 -0.8730979302777859,
 0.4096655293982669,
 0.14089314990910906,
 -0.07481023696510669,
 0.0,
 0.0,
 0.8845682464947819,
 0.0,
 1.0,
 0.9070018638862576,
 1.0,
 -0.8845682464947822,
 1.0,
 0.33900263784572826,
 0.9022509921112104,
 -1.0,
 -0.9092085160005584,
 1.0,
 0.9363909949752944,
 0.7486681672439952,
 0.5155242336366264,
 -1.0,
 0.0,
 -1.0,
 1.0,
 0.5903344706017328,
 0.4096655293982669,
 -0.26

#### Add polarity to the Pcampagna nodes

In [21]:
subset_pos['polarity'] = polarity_list
subset_pos.head(10)

,id,label,counts,counts_problematico,counts_positivo,counts_negativo,counts_ambiguo,polarity
0,0,collegamento,52,0,51,1,0,0.251332
1,1,continuo,66,0,61,5,0,-0.873098
2,2,tempo,167,1,153,13,0,1.000000
3,3,pastorello,1,0,1,0,0,0.000000
4,4,recuperare,12,0,12,0,0,-1.000000
5,5,pecorella,1,0,1,0,0,0.000000
6,6,smarrito,1,0,1,0,0,-0.543199
7,7,trovare,142,0,138,4,0,0.000000
8,8,fosso,1,0,1,0,0,-0.827441
9,9,localita,2,0,2,0,0,1.000000


In [22]:
subset_pos.to_csv(filename_nodes[1], index = False) ## save the positive 

In [23]:
polarity_list = []

for i in range(len(subset_neg)):
    if (i%2000 == 0): print(i, "/", len(subset_neg))
    word = subset_neg.label[i]    
    polarity_list.append(posts[posts.loc[:,'lemma'] == word].polarity.tolist()[0])
polarity_list

0 / 10985
2000 / 10985
4000 / 10985
6000 / 10985
8000 / 10985
10000 / 10985


[1.0,
 0.4675002450093737,
 0.4096655293982669,
 -0.4096655293982672,
 1.0,
 -0.5471903285814137,
 1.0,
 0.8730979302777857,
 0.18066894120346624,
 0.2513318327560048,
 -0.8193310587965339,
 0.18066894120346624,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 0.0,
 1.0,
 -0.8193310587965339,
 -0.4096655293982672,
 1.0,
 0.5903344706017328,
 0.0,
 -0.7486681672439954,
 -0.8378456410227744,
 0.8591068500908909,
 0.0,
 0.0,
 -0.7807598457092364,
 -0.07481023696510669,
 -0.31191652150947746,
 1.0,
 0.0,
 0.0,
 -1.0,
 -1.0,
 -1.0,
 0.0,
 -1.0,
 0.0,
 -0.04713547548136576,
 0.0,
 0.31191652150947724,
 -1.0,
 0.0,
 1.0,
 0.3229878906786693,
 0.2337952069749858,
 0.0,
 0.0,
 0.7897261731549863,
 -0.2513318327560048,
 -0.4096655293982672,
 0.0,
 0.3390026378457284,
 0.4096655293982669,
 -1.0,
 0.7215820509077442,
 1.0,
 1.0,
 -0.5903344706017328,
 0.6880834784905228,
 -1.0,
 -1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 -0.4096655293982672,
 0.4096655293982669,
 0.0,
 0.0,
 -0.7897261731549865,
 -0.7215820509077442,
 -0.25133183

In [24]:
subset_neg['polarity'] = polarity_list
subset_neg.head(10)


,id,label,counts,counts_problematico,counts_positivo,counts_negativo,counts_ambiguo,polarity
0,0,musulmano,15,5,3,6,1,1.000000
1,1,pensare,113,7,54,52,0,0.467500
2,2,portare,148,17,81,50,0,0.409666
3,3,guerra,76,7,32,33,4,-0.409666
4,4,santo,18,2,6,10,0,1.000000
5,5,casa,250,24,107,117,2,-0.547190
6,6,arrivare,121,22,65,33,1,1.000000
7,7,momento,51,1,30,20,0,0.873098
8,8,prendere,139,11,63,65,0,0.180669
9,9,provvedimento,36,2,18,16,0,0.251332


In [25]:
subset_neg.to_csv(filename_nodes[0], index = False)

## LIWC

In [26]:
###Load the posts LIWC file
#obtain the useful dataset
# LIWC2015 Results (Posts_lemmas_unique.csv)
LIWC_dict = pd.read_csv('LIWC2015 Results (Posts_lemmas_unique.csv).csv')
LIWC_dict = LIWC_dict.loc[0::,:].drop('A', axis = 1).dropna()
LIWC_dict

,B,WC,WPS,Sixltr,Dic,pronomi,Io,Noi,Se,Tu,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,musulmano,1,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,pensare,1,1.0,100.0,100.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,portare,1,1.0,100.0,100.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,guerra,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,santo,1,1.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17577,#ricorso,1,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
17578,xadiscussione,1,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17579,xadibattito,1,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17580,xaregole,1,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
###Load the csv nodes file for p_campagna pos & Neg with polarity
subset_pos= pd.read_csv('posts_subsets/NODES_Positiva_prating.csv')
subset_neg = pd.read_csv('posts_subsets/NODES_Neg-comp_prating.csv')
subset_pos.head()

,id,label,counts,counts_problematico,counts_positivo,counts_negativo,counts_ambiguo,polarity
0,0,collegamento,52,0,51,1,0,0.251332
1,1,continuo,66,0,61,5,0,-0.873098
2,2,tempo,167,1,153,13,0,1.000000
3,3,pastorello,1,0,1,0,0,0.000000
4,4,recuperare,12,0,12,0,0,-1.000000


In [28]:
subset_neg.head()

,id,label,counts,counts_problematico,counts_positivo,counts_negativo,counts_ambiguo,polarity
0,0,musulmano,15,5,3,6,1,1.000000
1,1,pensare,113,7,54,52,0,0.467500
2,2,portare,148,17,81,50,0,0.409666
3,3,guerra,76,7,32,33,4,-0.409666
4,4,santo,18,2,6,10,0,1.000000


In [29]:
#list the variables one may want to keep
columns_to_keep = ['B', 'Emo_Neg',  'Ansia',  'Rabbia',  'Tristez', 'parolac']
att_to_keep = ['Emo_Neg',  'Ansia',  'Rabbia',  'Tristez', 'parolac']

In [30]:
LIWC_dict_sub = LIWC_dict.loc[:,columns_to_keep].dropna()
LIWC_dict_sub

,B,Emo_Neg,Ansia,Rabbia,Tristez,parolac
0,musulmano,0.0,0.0,0.0,0.0,0.0
1,pensare,0.0,0.0,0.0,0.0,0.0
2,portare,0.0,0.0,0.0,0.0,0.0
3,guerra,0.0,0.0,0.0,0.0,0.0
4,santo,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
17577,#ricorso,0.0,0.0,0.0,0.0,0.0
17578,xadiscussione,0.0,0.0,0.0,0.0,0.0
17579,xadibattito,0.0,0.0,0.0,0.0,0.0
17580,xaregole,0.0,0.0,0.0,0.0,0.0


#### Add LIWC attributes to the subset

In [31]:
####update the negavtive campagna
for i, word in enumerate(subset_neg.label):
    subsub = LIWC_dict_sub[LIWC_dict_sub.B == word].loc[:,att_to_keep]
    for j in att_to_keep: 
        subset_neg.loc[i, j] = subsub.loc[:,j].tolist()[0]
        
    if i%2000 == 0: print(i, "/", len(subset_neg.label))
print("Done.")

0 / 10985
2000 / 10985
4000 / 10985
6000 / 10985
8000 / 10985
10000 / 10985
Done.


In [32]:
subset_neg.to_csv('posts_subsets/NODES_prating_LIWC_neg.csv', index=False)
subset_neg

,id,label,counts,counts_problematico,counts_positivo,counts_negativo,counts_ambiguo,polarity,Emo_Neg,Ansia,Rabbia,Tristez,parolac
0,0,musulmano,15,5,3,6,1,1.000000,0.0,0.0,0.0,0.0,0.0
1,1,pensare,113,7,54,52,0,0.467500,0.0,0.0,0.0,0.0,0.0
2,2,portare,148,17,81,50,0,0.409666,0.0,0.0,0.0,0.0,0.0
3,3,guerra,76,7,32,33,4,-0.409666,0.0,0.0,0.0,0.0,0.0
4,4,santo,18,2,6,10,0,1.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10980,10981,quindicenne,1,1,0,0,0,0.000000,0.0,0.0,0.0,0.0,0.0
10981,10982,#vigliacco,1,0,0,1,0,-0.915243,0.0,0.0,0.0,0.0,0.0
10982,10983,indugiare,1,0,1,0,0,-1.000000,0.0,0.0,0.0,0.0,0.0
10983,10984,brambilla,1,0,1,0,0,0.000000,0.0,0.0,0.0,0.0,0.0


In [33]:
for i, word in enumerate(subset_pos.label):
    subsub = LIWC_dict_sub[LIWC_dict_sub.B == word].loc[:,att_to_keep]
    for j in att_to_keep: 
        subset_pos.loc[i, j] = subsub.loc[:,j].tolist()[0]
        
    if i%1000 == 0: print(i, "/", len(subset_pos.label))
        
print("Done.")

0 / 12947
1000 / 12947
2000 / 12947
3000 / 12947
4000 / 12947
5000 / 12947
6000 / 12947
7000 / 12947
8000 / 12947
9000 / 12947
10000 / 12947
11000 / 12947
12000 / 12947
Done.


In [34]:
subset_pos.to_csv('posts_subsets/NODES_prating_LIWC_pos.csv', index=False)
subset_pos

,id,label,counts,counts_problematico,counts_positivo,counts_negativo,counts_ambiguo,polarity,Emo_Neg,Ansia,Rabbia,Tristez,parolac
0,0,collegamento,52,0,51,1,0,0.251332,0.0,0.0,0.0,0.0,0.0
1,1,continuo,66,0,61,5,0,-0.873098,0.0,0.0,0.0,0.0,0.0
2,2,tempo,167,1,153,13,0,1.000000,0.0,0.0,0.0,0.0,0.0
3,3,pastorello,1,0,1,0,0,0.000000,0.0,0.0,0.0,0.0,0.0
4,4,recuperare,12,0,12,0,0,-1.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12942,12942,#ricorso,1,0,1,0,0,-0.452810,0.0,0.0,0.0,0.0,0.0
12943,12943,xadiscussione,1,0,1,0,0,0.251332,0.0,0.0,0.0,0.0,0.0
12944,12944,xadibattito,2,0,2,0,0,1.000000,0.0,0.0,0.0,0.0,0.0
12945,12945,xaregole,1,0,1,0,0,0.000000,0.0,0.0,0.0,0.0,0.0
